# Panini-Meru Training Analysis

This notebook analyzes the training metrics for Panini-Meru runs.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import glob

In [ ]:
# Configuration
OUTPUT_DIR = "../output" # Adjust relative to notebook location

# Find all runs
runs = glob.glob(os.path.join(OUTPUT_DIR, "*"))
print(f"Found {len(runs)} runs:")
for r in runs:
    print(f" - {os.path.basename(r)}")

In [ ]:
def load_metrics(run_path):
    metrics_file = os.path.join(run_path, "metrics.jsonl")
    if not os.path.exists(metrics_file):
        print(f"No metrics found for {run_path}")
        return None
    
    data = []
    with open(metrics_file, 'r') as f:
        for line in f:
            try:
                data.append(json.loads(line))
            except:
                pass
                
    return pd.DataFrame(data)

In [ ]:
# Select a run (default: latest)
if runs:
    runs.sort(key=os.path.getmtime, reverse=True)
    latest_run = runs[0]
    run_name = os.path.basename(latest_run)
    print(f"Analyzing: {run_name}")

    df = load_metrics(latest_run)
    if df is not None and not df.empty:
        display(df.head())
    else:
        print("No metrics data found yet.")
else:
    print("No runs found at all.")

In [ ]:
# Plot Loss
if df is not None and not df.empty:
    plt.figure(figsize=(10, 6))
    plt.plot(df['step'], df['loss'], label='Training Loss')
    plt.xlabel('Steps')
    plt.ylabel('Loss')
    plt.title(f'Training Loss - {run_name}')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.show()

## Moving Average
Smooth out the noise to see the trend.

In [ ]:
if df is not None and not df.empty:
    WINDOW = 10
    df['loss_ma'] = df['loss'].rolling(window=WINDOW).mean()

    plt.figure(figsize=(10, 6))
    plt.plot(df['step'], df['loss'], alpha=0.3, label='Raw Loss')
    plt.plot(df['step'], df['loss_ma'], color='red', label=f'{WINDOW}-Step MA')
    plt.xlabel('Steps')
    plt.ylabel('Loss')
    plt.title(f'Smoothed Training Loss - {run_name}')
    plt.grid(True)
    plt.legend()
    plt.show()